In [1]:
# Need to be run on Virtual Machine

import dinopy
import pandas as pd
import tables
import numpy as np
import glob

# Get all .fasta files in the specified directory
fasta_files = glob.glob('../data/interim/finalData/*.fasta')

# create empty DataFrame
sample_names = [i.split("/")[-1].split(".")[0] for i in fasta_files]
df = pd.DataFrame(dtype=np.uint64)

# process each .fasta file
for i in range(len(fasta_files)):
    sample_name = sample_names[i]
    seqs = dinopy.FastaReader(fasta_files[i])
    
    # create a dictionary to store counts for each sequence
    seq_counts = {}
    for entry in seqs.entries():
        seq = entry.sequence.decode()
        value = np.uint64(entry.name.decode().split("size=")[1].split(";")[0])
        seq_counts[seq] = seq_counts.get(seq, 0) + value
    
    # convert the dictionary to a DataFrame and add it to the main DataFrame
    df_sample = pd.DataFrame.from_dict(seq_counts, orient='index', columns=[sample_name])
    df = pd.concat([df, df_sample], axis=1).fillna(0).astype(np.uint64)

# save to file
df.index.name = "sequences"
df.to_hdf('../data/processed/output.h5', key='df', mode='w')
df.to_csv('../data/processed/output.csv')